# Comparison between MLP and CNN
In this experiment, our primary focus is on understanding the main properties of Convolutional Neural Networks (CNNs). In this regard, we compare a CNN model against a traditional MLP model in terms of architectural components and performance.
Both models are designed for binary classification of the network traffic as benign or malicious. Therefore, they return a value between 0 and 1, which is the probability of the input flow of being malicious. 


| <img src="./mlp-cnn.png" width="100%">  |
|--|
| MLP and CNN architectures: n=100, f=20, m=100, k=100|

We will use a dataset of benign and various DDoS attacks from the CIC-DDoS2019 dataset (https://www.unb.ca/cic/datasets/ddos-2019.html).
The network traffic has been previously pre-processed in a way that packets are grouped in bi-directional traffic flows using the 5-tuple (source IP, destination IP, source Port, destination Port, protocol). Each flow is represented as a 100x20 array, where the rows are the packets of the flow in chronological order, while each column is a packet-level feature in the following order:

| Feature nr.         | Feature Name |
|---------------------|---------------------|
| 00 | timestamp (IAT) | 
| 01 | packet_length (bytes)| 
| 02 | IP_flags_df (0/1) |
| 03 | IP_flags_mf (0/1) |
| 04 | IP_flags_rb (0/1) | 
| 05 | IP_frag_off (0/1) |
| 06 | protocols (integer) |
| 07 | TCP_length (bytes) |
| 08 | TCP_flags_ack (0/1) |
| 09 | TCP_flags_cwr (0/1) |
| 10 | TCP_flags_ece (0/1) |
| 11 | TCP_flags_fin (0/1) |
| 12 | TCP_flags_push (0/1) |
| 13 | TCP_flags_res (0/1) |
| 14 | TCP_flags_reset (0/1) |
| 15 | TCP_flags_syn (0/1) |
| 16 | TCP_flags_urg (0/1) |
| 17 | TCP_window_size (bytes) |
| 18 | UDP_length (bytes) |
| 19 | ICMP_type (code) |

In [ ]:
# Author: Roberto Doriguzzi-Corin
# Project: Course on Network Intrusion and Anomaly Detection with Machine Learning
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import time
import numpy as np
import pprint
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, GlobalMaxPooling2D, Flatten
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import set_random_seed
from util_functions import *

EPOCHS = 100
TEST_ITERATIONS=100

# disable GPUs for test reproducibility
tf.config.set_visible_devices([], 'GPU')

SEED = 0
np.random.seed(SEED)
set_random_seed(SEED)

DATASET_FOLDER = "./DOS2019"

In [ ]:
def report_results(Y_true, Y_pred, model_name, data_source, prediction_time):
    ddos_rate = '{:04.3f}'.format(sum(Y_pred) / Y_pred.shape[0])

    if Y_true is not None and len(Y_true.shape) > 0:  # if we have the labels, we can compute the classification accuracy
        Y_true = Y_true.reshape((Y_true.shape[0], 1))
        accuracy = accuracy_score(Y_true, Y_pred)

        f1 = f1_score(Y_true, Y_pred)
        tn, fp, fn, tp = confusion_matrix(Y_true, Y_pred, labels=[0, 1]).ravel()
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        fnr = fn / (fn + tp)
        tpr = tp / (tp + fn)

        row = {'Model': model_name, 'Time': '{:04.3f}'.format(prediction_time),
               'Samples': Y_pred.shape[0], 'DDOS%': ddos_rate, 'Accuracy': '{:05.4f}'.format(accuracy), 'F1Score': '{:05.4f}'.format(f1),
               'TPR': '{:05.4f}'.format(tpr), 'FPR': '{:05.4f}'.format(fpr), 'TNR': '{:05.4f}'.format(tnr), 'FNR': '{:05.4f}'.format(fnr), 'Source': data_source}

    pprint.pprint(row, sort_dicts=False)

In [ ]:
def plot_history(history):
    # Plot training and validation accuracy over epochs
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Training and Validation Accuracy')
    plt.show()

    # Plot training and validation loss over epochs
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    #plt.ylim([0, 2])
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.show()

# MLP model definition and training
In the following cell, we define and compile an MLP model. Then we train it for a certain number of epochs and we measure the training time.

In [ ]:
X_train, y_train = load_dataset(DATASET_FOLDER + "/*" + '-train.hdf5')
X_val, y_val = load_dataset(DATASET_FOLDER + "/*" + '-val.hdf5')

mlp_model = Sequential()
mlp_model.add(Flatten(input_shape=X_train.shape[1:]))
mlp_model.add(Dense(100, activation='relu'))
mlp_model.add(Dense(1, activation='sigmoid'))

# Compile the model
mlp_model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Print a summary of the model architecture
mlp_model.summary()

# Train the model
start_time = time.time()
history = mlp_model.fit(X_train, y_train, epochs=EPOCHS, validation_data=(X_val, y_val), batch_size=16, verbose=1)
stop_time = time.time()

print ("Training time/epoch (sec): ", (stop_time-start_time)/EPOCHS)

plot_history(history)

# CNN model definition and training
In the following cell, we define and compile an CNN model. Then we train it for a certain number of epochs and we measure the training time.

In [ ]:
X_train, y_train = load_dataset(DATASET_FOLDER + "/*" + '-train.hdf5',channels=True)
X_val, y_val = load_dataset(DATASET_FOLDER + "/*" + '-val.hdf5',channels=True)

cnn_model = Sequential()
cnn_model.add(Conv2D(filters=100, kernel_size=(3, X_train.shape[2]), input_shape=X_train.shape[1:], data_format='channels_last', activation='relu', padding='same', strides=(1, 1)))
cnn_model.add(GlobalMaxPooling2D())
cnn_model.add(Flatten())
cnn_model.add(Dense(1, activation='sigmoid'))

# Compile the model
cnn_model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Print a summary of the model architecture
cnn_model.summary()

# Train the model
start_time = time.time()
history = cnn_model.fit(X_train, y_train, epochs=int(EPOCHS/5), validation_data=(X_val, y_val), batch_size=16, verbose=1)
stop_time = time.time()

print ("Training time/epoch (sec): ", (stop_time-start_time)/(EPOCHS/5))

plot_history(history)

# Inference time comparison

In [ ]:
# MLP model
X_test, y_test = load_dataset(DATASET_FOLDER + "/*" + '-test.hdf5')

pt0 = time.time()
for i in range(TEST_ITERATIONS):
    Y_pred = np.squeeze(mlp_model.predict(X_test, batch_size=16) > 0.5,axis=1)
pt1 = time.time()
prediction_time = pt1 - pt0

report_results(np.squeeze(y_test), Y_pred,  'MLP', '', prediction_time/TEST_ITERATIONS)

In [ ]:
# CNN model
X_test, y_test = load_dataset(DATASET_FOLDER + "/*" + '-test.hdf5',channels=True)

pt0 = time.time()
for i in range(TEST_ITERATIONS):
    Y_pred = np.squeeze(cnn_model.predict(X_test, batch_size=16) > 0.5,axis=1)
pt1 = time.time()
prediction_time = pt1 - pt0

report_results(np.squeeze(y_test), Y_pred,  'CNN', '', prediction_time/TEST_ITERATIONS)